In [1]:
import pickle
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
import re
import multiprocess
import ast
import warnings
warnings.filterwarnings("ignore")

In [2]:
# datetime of each financial report
with open('../fp_url_htm/company_dt_dic_final.pkl','rb') as f:
    company_dt_dic = pickle.load(f)
    
# htm or txt url of each financial report
with open('../fp_url_htm/company_fp_dic_final.pkl','rb') as f:
    company_fp_dic = pickle.load(f)
    
# mark of each financial report
# 0 : crawl from raw 1(htm or txt)  1: crawl from original txt
with open('../fp_url_htm/company_mk_dic_final.pkl','rb') as f:
    company_mk_dic = pickle.load(f)

In [3]:
score = pd.read_csv('../score.csv')

In [2]:
def filter_scoresum(df,threshold):
    scoresum_list = []
    df_score = df['score'].values
    for score in df_score:
        scoresum_list.append(sum(ast.literal_eval(score)))
    df['scoresum'] = scoresum_list
    return df[df['scoresum'] < threshold].iloc[:,0]

In [5]:
need_recrawl_index = filter_scoresum(score,100)

In [6]:
need_recrawl_index.values[1]

96

In [8]:
SP_return=pd.read_csv("SP_return.csv",index_col=0,parse_dates=True)
SP_price=(SP_return+1).cumprod()["sprtrn"]
SP_return.head()

,sprtrn
caldt,
1990-01-02,0.017799
1990-01-03,-0.002586
1990-01-04,-0.008613
1990-01-05,-0.009756
1990-01-08,0.004514


In [3]:
all_company_data=pd.read_csv("../sasa/all_company_data_ver2.csv")

In [4]:
all_company_data.shape

(282246, 6)

In [5]:
cutDataAndNeedtoCrawl = pd.read_csv('../sasa/cutDataAndNeedtoCrawl.csv')

In [7]:
cutDataAndNeedtoCrawl.head()

,company,date,return,bad,url,QK,text
0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q,item 2. management's discussion and analysis o...
1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q,item 2. management's discussion and analysis o...
2,A,2000-09-01,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q,NaN
3,A,2001-01-17,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K,NaN
4,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q,item 2. management's discussion and analysis o...


In [29]:
def get_CrawlIndex(cutDataAndNeedtoCrawl):
    QK = cutDataAndNeedtoCrawl['QK'].values
    texts = cutDataAndNeedtoCrawl['text'].values
    index = cutDataAndNeedtoCrawl.index
    crawl_index = []
    
    for i,types in enumerate(QK):
        if(types == 'Q'):
            if(type(texts[i]) != str):
                crawl_index.append(1)
            else:
                crawl_index.append(0)
        else:
            crawl_index.append(0)
    return crawl_index

In [30]:
need_recrawl_index = get_CrawlIndex(cutDataAndNeedtoCrawl)

In [31]:
need_recrawl_index = np.array(need_recrawl_index)
need_recrawl_index.shape

(282246,)

In [6]:
def cut_item2(input_data_):
    input_data, idx, need_recrawl_index = input_data_
    output=input_data.copy()
    output["text"]="NA"
    else_deal=[]
    for i in range(input_data.shape[0]):
        if(need_recrawl_index[i] == 1):
            try:
                index_now=input_data.index[i]
                url="https://www.sec.gov"+input_data["url"].iloc[i]
                res = requests.get(url)
                soup = BeautifulSoup(res.text, 'html.parser')
                soup_txt = soup.get_text()
                soup_txt=soup_txt.replace('\xa0',' ').replace('&nbsp;',' ').replace('&#160;',' ').replace('\n',' ').replace('\x93',' ').replace('\x94',' ').replace(':',' ')
                soup_txt=soup_txt.split(" ")
                while '' in soup_txt:
                    soup_txt.remove('')
                space=" "
                soup_txt=space.join(soup_txt)
                soup_txt=soup_txt.lower()

                pattern=re.compile("item 2. manage")
                result_iter1=pattern.finditer(soup_txt)
                result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("item 2 – manage")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("item 2.manage")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("item 2 - manage")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("item 2 — manage")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("item 2: manage")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("management's discussion and analysis")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("management’s discussion and analysis")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("managements' discussion and analysis")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("management's discussion of operations and financial condition")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                if len(result_item2) == 0:
                    pattern=re.compile("item 2. m anage")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item2 = [ m1.span() for m1 in result_iter1]
                #print(len(result_item2))

                pattern=re.compile("item 3. quant")
                result_iter1=pattern.finditer(soup_txt)
                result_item3 = [ m1.span() for m1 in result_iter1]
                if len(result_item3) == 0:
                    pattern=re.compile("item 3 – quant")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item3 = [ m1.span() for m1 in result_iter1]
                if len(result_item3) == 0:
                    pattern=re.compile("item 3.quant")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item3 = [ m1.span() for m1 in result_iter1]
                if len(result_item3) == 0:
                    pattern=re.compile("item 3 - quant")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item3 = [ m1.span() for m1 in result_iter1]
                if len(result_item3) == 0:
                    pattern=re.compile("item 3 — quant")
                    result_iter1=pattern.finditer(soup_txt)
                    result_item3 = [ m1.span() for m1 in result_iter1]
                #print(len(result_item3))

                pattern=re.compile("part ii")
                result_iter1=pattern.finditer(soup_txt)
                result_part2=[ m1.span() for m1 in result_iter1]
                #print(len(result_part2))

                if len(result_item2) == 1:
                    if len(result_item3) == 1:
                        output["text"].loc[index_now]=soup_txt[result_item2[0][0]:result_item3[0][0]]
                    elif len(result_item3) == 2:
                        output["text"].loc[index_now]=soup_txt[result_item2[0][0]:result_item3[1][0]]
                    else:    
                        if len(result_part2) != 0:
                            output["text"].loc[index_now]=soup_txt[result_item2[0][0]:result_part2[-1][0]]
                        else:
                            output["text"].loc[index_now]=soup_txt[result_item2[0][0]:]


                elif len(result_item2) == 2:
                    if len(result_item3) == 2:
                        #(2,2)
                        output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[1][0]]
                    elif len(result_item3) == 1:
                        #(2,1)
                        output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[0][0]]
                    else:
                        if len(result_part2) >= 2:
                            output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_part2[1][0]]
                        elif len(result_part2) == 1:
                            output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_part2[0][0]]
                        else:
                            output["text"].loc[index_now]=soup_txt[result_item2[1][0]:]


                elif len(result_item2) == 3:
                    if len(result_item3) == 1:
                        output["text"].loc[index_now]=soup_txt[result_item2[-1][0]:result_item3[0][0]]
                    elif len(result_item3) >= 2:
                        output["text"].loc[index_now]=soup_txt[result_item2[-1][0]:result_item3[-1][0]]
                    else:  
                        if len(result_part2) != 0:
                            output["text"].loc[index_now]=soup_txt[result_item2[-1][0]:result_part2[-1][0]]
                        else:
                            output["text"].loc[index_now]=soup_txt[result_item2[-1][0]:]

                elif len(result_item2) > 3:
                    if len(result_item3) == 1:
                        output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[0][0]]
                    elif len(result_item3) >= 2:
                        output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_item3[1][0]]
                    else:  
                        if len(result_part2) != 0:
                            output["text"].loc[index_now]=soup_txt[result_item2[1][0]:result_part2[-1][0]]
                        else:
                            output["text"].loc[index_now]=soup_txt[result_item2[1][0]:]               
                else:
                    print('*****************wronggggg*****************')
                    print(url)
                    print(i)
                    print("result_item2 = "+str(len(result_item2)))
                    print("result_item3 = "+str(len(result_item3)))
                    print("result_part2 = "+str(len(result_part2)))
                    else_deal.append(i)

            except:
                print("----------------unsolved--------------")
                print(url)
                print(i)

        if(i % 5000 == 0 and i != 0):
            print(i)
            multiple_fix = int(i/10000)
            output.to_csv(f'cut_data_{idx}_{multiple_fix}_10Q_9.csv')
            
#     output.to_csv(f'cut_data_{idx}_!0Q_9.csv')
#     output.to_csv(f'cut_data_{idx}_!0Q_9.csv')
    return output

In [36]:
all_company_data.head()

,company,date,return,bad,url,QK
0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q
1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q
2,A,2000-09-01,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q
3,A,2001-01-17,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K
4,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q


In [37]:
# need_recrawl_index = need_recrawl_index.values
length = int(need_recrawl_index.shape[0]/8)
need_recrawl_index0, need_recrawl_index1,need_recrawl_index2,need_recrawl_index3,\
need_recrawl_index4,need_recrawl_index5,need_recrawl_index6,need_recrawl_index7 \
= need_recrawl_index[:length], need_recrawl_index[length:length*2], need_recrawl_index[length*2:length*3],\
need_recrawl_index[length*3:length*4], need_recrawl_index[length*4:length*5], need_recrawl_index[length*5:length*6],\
need_recrawl_index[length*6:length*7] , need_recrawl_index[length*7:] 
    
    

In [38]:
length = int(all_company_data.shape[0]/8)
company_data0, company_data1, company_data2, company_data3,company_data4,company_data5,company_data6,company_data7 = \
all_company_data.iloc[:length,:], all_company_data.iloc[length:length*2,:],all_company_data.iloc[length*2:length*3,:]\
,all_company_data.iloc[length*3:length*4,:],all_company_data.iloc[length*4:length*5,:],all_company_data.iloc[length*5:length*6,:],all_company_data.iloc[length*6:length*7,:]\
,all_company_data.iloc[length*7:,:]

works = [(company_data0,0,need_recrawl_index0),(company_data1,1,need_recrawl_index1),(company_data2,2,need_recrawl_index2),\
         (company_data3,3,need_recrawl_index3),(company_data4,4,need_recrawl_index4),(company_data5,5,need_recrawl_index5)\
        ,(company_data6,6,need_recrawl_index6),(company_data7,7,need_recrawl_index7)]

# works


In [28]:
company_data2

,company,date,return,bad,url,QK
70560,DAKT,2011-09-02,0.969126,0.0,/Archives/edgar/data/915779/000091577911000045...,Q
70561,DAKT,2011-12-02,0.947748,0.0,/Archives/edgar/data/915779/000091577911000068...,Q
70562,DAKT,2012-03-02,0.788054,0.0,/Archives/edgar/data/915779/000091577912000006...,Q
70563,DAKT,2012-06-13,1.427924,0.0,/Archives/edgar/data/915779/000091577912000013...,K
70564,DAKT,2012-08-31,0.996909,0.0,/Archives/edgar/data/915779/000091577912000031...,Q


In [ ]:
if __name__ == '__main__':
    ans = []
    pool = multiprocess.Pool(processes=8)
    ans.append(pool.map(cut_item2,works))

*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/27419/000110465904026876/a04-10061_110q.htm
239
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/27419/000110465904038358/a04-14165_110q.htm
240
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/320187/000032018799000012/0000320187-99-000012.txt
191
result_item2 = 0
result_item3 = 0
result_part2 = 0
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/320187/000032018700000003/0000320187-00-000003.txt
192
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/ix?doc=/Archives/edgar/data/27904/000002790416000029/dal930201610q.htm
106
result_item2 = 0
result_item3 = 0
result_part2 = 0
*****************wronggggg*****************
https://www.sec.gov/

662
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1005409/000100540901500022/exh11301.txt
922
result_item2 = 0
result_item3 = 0
result_part2 = 0
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/50725/000110465904023413/a04-9109_110q.htm
905
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/50725/000110465905004884/a05-2953_110q.htm
907
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/50725/000110465906007384/a06-4534_110q.htm
911
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/50725/000110465908008845/a08-5044_110q.htm
919
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg************

92
result_item2 = 0
result_item3 = 2
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/850033/000095015207006657/l27414ae10vq.htm
93
result_item2 = 0
result_item3 = 1
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/850033/000095015207008842/l28739ae10vq.htm
94
result_item2 = 0
result_item3 = 2
result_part2 = 4
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/850033/000095015208003775/l31532ae10vq.htm
96
result_item2 = 0
result_item3 = 2
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/850033/000095015208006237/l32814ae10vq.htm
97
result_item2 = 0
result_item3 = 2
result_part2 = 5
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/850033/000095015208008935/l34461ae10vq.htm
98
result_item2 = 0
result_item3 = 1
result_part2 = 1
*****************wronggggg***************

result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/0000912057-94-003805.txt
1903
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/0000912057-95-003401.txt
1905
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/0000912057-95-006172.txt
1906
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/0000912057-96-009641.txt
1907
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/0000912057-96-017491.txt
1908
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/0000912057-96-025968.txt
19

1960
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/000110465910056473/a10-16055_110q.htm
1961
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/000110465911026773/a11-5994_110q.htm
1963
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/000110465911043664/a11-13772_110q.htm
1964
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/000110465911061064/a11-25611_110q.htm
1965
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1800/000110465912034444/a12-7133_110q.htm
1967
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****

...

https://www.sec.gov/Archives/edgar/data/1105705/000119312513323650/d553650d10q.htm
9784
result_item2 = 0
result_item3 = 0
result_part2 = 4
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1105705/000119312513429610/d612918d10q.htm
9785
result_item2 = 0
result_item3 = 0
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1105705/000119312514170378/d708029d10q.htm
9787
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1105705/000119312514297593/d756102d10q.htm
9788
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1105705/000119312514397875/d802289d10q.htm
9789
result_item2 = 0
result_item3 = 0
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1105705/000119312515155546/d909150d10q.htm
97

9932
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/0000930661-95-000448.txt
9933
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/0000930661-96-000435.txt
9935
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/716133/0000912057-97-026690.txt
7037
result_item2 = 0
result_item3 = 0
result_part2 = 0
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/0000930661-96-001009.txt
9936
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/0000930661-97-001293.txt
9937
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/

https://www.sec.gov/Archives/edgar/data/710752/000119312512215217/d337305d10q.htm
9987
result_item2 = 0
result_item3 = 2
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/000119312512340636/d383390d10q.htm
9988
result_item2 = 0
result_item3 = 2
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/000119312512457218/d429818d10q.htm
9989
result_item2 = 0
result_item3 = 2
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/000119312513204587/d523323d10q.htm
9991
result_item2 = 0
result_item3 = 2
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/000119312513324062/d572797d10q.htm
9992
result_item2 = 0
result_item3 = 2
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/710752/000119312513424954/d616290d10q.htm
9993
res

11182
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/94049/000119312510171622/d10q.htm
11183
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/94049/000119312510247355/d10q.htm
11184
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/94049/000119312511202000/d10q.htm
11187
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/94049/000119312511286131/d239524d10q.htm
11188
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/94049/000119312512448218/d398225d10q.htm
11192
result_item2 = 0
result_item3 = 0
result_part2 = 1
*****************wronggggg*****************
https://ww

7146
result_item2 = 0
result_item3 = 0
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1098972/000095013502003707/b43659ane10vq.htm
7147
result_item2 = 0
result_item3 = 0
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1098972/000095013502005044/b44475aie10vq.htm
7148
result_item2 = 0
result_item3 = 0
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1098972/000095013503003090/b46494age10vq.htm
7150
result_item2 = 0
result_item3 = 0
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1098972/000095013503004404/b47133aie10vq.htm
7151
result_item2 = 0
result_item3 = 0
result_part2 = 3
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/1098972/000119312503080168/d10q.htm
7152
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg**

In [ ]:
output = pd.concat([ans[0][0],ans[0][1]],axis = 0)

In [ ]:
output = cut_item2((all_company_data,0,need_recrawl_index))

In [13]:
len(need_recrawl_index)

18732

In [17]:
test = pd.read_csv('cut_data_0_all_2.csv')

In [19]:
test.head(50)

,Unnamed: 0,company,date,return,bad,url,text
0,0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,NaN
1,1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,NaN
2,2,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,NaN
3,3,A,2001-06-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,NaN
4,4,A,2002-03-06,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,NaN
5,5,A,2002-06-05,0.560176,1.0,/Archives/edgar/data/1090872/00008916180200270...,NaN
6,6,A,2003-03-12,1.192572,0.0,/Archives/edgar/data/1090872/00008916180300121...,NaN
7,7,A,2003-06-04,1.312234,0.0,/Archives/edgar/data/1090872/00008916180300285...,NaN
8,8,A,2004-03-04,0.805133,1.0,/Archives/edgar/data/1090872/00011931250403472...,NaN
9,9,A,2004-06-03,0.823663,0.0,/Archives/edgar/data/1090872/00011931250409832...,NaN


In [44]:
origin = pd.read_csv('cut_data_all.csv')

In [27]:
need_recrawl_index.values[0]

95

In [ ]:
for index in need_recrawl_index.values:
    temp = test.iloc[index,6]
    if temp:
        origin.iloc[index,6] = temp

In [50]:
origin.to_csv('cut_data_all.csv',index=False)

In [51]:
all_company_data_v2 = pd.read_csv("../sasa/all_company_data_ver2.csv")

In [57]:
aa = origin.loc[:,['company','date','text']]

In [58]:
aa

,company,date,text
0,A,2000-03-15,item 2. management's discussion and analysis o...
1,A,2000-06-12,item 2. management's discussion and analysis o...
2,A,2001-03-19,item 2. management's discussion and analysis o...
3,A,2001-06-14,item 2. management's discussion and analysis o...
4,A,2002-03-06,item 2. managements discussion and analysis o...
5,A,2002-06-05,item 2. managements discussion and analysis o...
6,A,2003-03-12,item 2. managements discussion and analysis o...
7,A,2003-06-04,item 2. managements discussion and analysis o...
8,A,2004-03-04,item 2. managements discussion and analysis o...
9,A,2004-06-03,item 2. managements discussion and analysis o...


In [52]:
all_company_data_v2

,company,date,return,bad,url,QK
0,A,2000-03-15,0.680463,1.0,/Archives/edgar/data/1090872/00009120570001170...,Q
1,A,2000-06-12,0.937685,0.0,/Archives/edgar/data/1090872/00009120570002820...,Q
2,A,2000-09-01,0.893940,0.0,/Archives/edgar/data/1090872/00009120570004000...,Q
3,A,2001-01-17,0.557929,1.0,/Archives/edgar/data/1090872/00010958110100022...,K
4,A,2001-03-19,0.947393,0.0,/Archives/edgar/data/1090872/00010958110100163...,Q
5,A,2001-06-14,0.821604,0.0,/Archives/edgar/data/1090872/00008916180150118...,Q
6,A,2001-09-14,1.462535,0.0,/Archives/edgar/data/1090872/00010958110150492...,Q
7,A,2002-01-22,1.034088,0.0,/Archives/edgar/data/1090872/00008916180200018...,K
8,A,2002-03-06,0.741458,0.0,/Archives/edgar/data/1090872/00008916180200104...,Q
9,A,2002-06-05,0.560176,1.0,/Archives/edgar/data/1090872/00008916180200270...,Q


In [68]:
result = pd.merge(all_company_data_v2, aa,how='left')

In [70]:
result.to_csv('cutDataAndNeedtoCrawl.csv',index = False)

In [2]:
extra_all_company_data = pd.read_csv('extra_all_company_data.csv')

In [3]:
extra_all_company_data

,company,date,adjust_datetime,return,bad,url,QK
0,AAPL,1994-12-13,1994-12-13 00:00:00,0.993610,0.0,/Archives/edgar/data/320193/0000320193-94-0000...,K
1,AAPL,1994-12-30,1994-12-30 00:00:00,0.973724,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,Q
2,AAPL,1995-03-31,1995-03-31 00:00:00,1.330258,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,Q
3,AAPL,1995-12-19,1995-12-19 00:00:00,0.918606,0.0,/Archives/edgar/data/320193/0000320193-95-0000...,K
4,AAPL,1995-12-29,1995-12-29 00:00:00,0.803963,1.0,/Archives/edgar/data/320193/0000320193-96-0000...,Q
5,AAPL,1996-03-29,1996-03-29 00:00:00,0.937823,0.0,/Archives/edgar/data/320193/0000320193-96-0000...,Q
6,AAPL,1996-12-19,1996-12-19 00:00:00,0.664865,1.0,/Archives/edgar/data/320193/0000320193-96-0000...,K
7,AAPL,1996-12-27,1996-12-27 00:00:00,0.791664,1.0,/Archives/edgar/data/320193/0000320193-97-0000...,Q
8,AAPL,1997-03-28,1997-03-28 00:00:00,0.824501,1.0,/Archives/edgar/data/320193/0000320193-97-0000...,Q
9,AAPL,1997-12-05,1997-12-05 00:00:00,0.939914,0.0,/Archives/edgar/data/320193/0001047469-97-0069...,K


In [11]:
need_to_crawl_index = (extra_all_company_data['QK']=='Q').astype(int)

In [12]:
work = [extra_all_company_data, 0, need_to_crawl_index]

In [13]:
output = cut_item2(work)

*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/34408/000110465904000428/a04-1078_110q.htm
783
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/34408/000110465904009705/a04-4202_110q.htm
784
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/34408/000110465905000479/a05-1066_110q.htm
786
result_item2 = 0
result_item3 = 0
result_part2 = 2
*****************wronggggg*****************
https://www.sec.gov/ix?doc=/Archives/edgar/data/707549/000070754918000125/lrcx10q1q2019document.htm
962
result_item2 = 0
result_item3 = 0
result_part2 = 0
*****************wronggggg*****************
https://www.sec.gov/Archives/edgar/data/95585/000117152007000140/eps2453.htm
1125
result_item2 = 0
result_item3 = 0
result_part2 = 3


In [15]:
output.to_csv('extra_Q.csv',index = False)